<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/Copy_of_solaris_tf20_base_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2.1 Install check

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# -*- coding: utf-8 -*-
# 텐서플로우 설치 체크 - 텐서플로우가 제대로 설치되었는지 체크해봅시다.

import tensorflow as tf

hello = tf.constant('Hello, TensorFlow!')
print(hello.numpy())
#'Hello, TensorFlow!'

a = tf.constant(10)
b = tf.constant(32)
print((a + b).numpy())
#42

b'Hello, TensorFlow!'
42


3.1 Graph example

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# 그래프 노드를 정의하고 출력합니다.
# 출력값 : <tf.Tensor: id=0, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=1, shape=(), dtype=float32, numpy=4.0>
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # 암시적으로 tf.float32 타입으로 선언될 것입니다.
print(node1, node2)

# 그래프를 실행합니다.
# 출력값 : [3.0, 4.0]
print(node1.numpy(), node2.numpy())

# 두개의 노드의 값을 더하는 연산을 수행하는 node3을 정의합니다.
# 출력값:
# node3: <tf.Tensor: id=2, shape=(), dtype=float32, numpy=7.0>
# node3.numpy(): 7.0
node3 = tf.add(node1, node2)
print("node3:", node3)
print("node3.numpy():", node3.numpy())

tf.Tensor(3.0, shape=(), dtype=float32) tf.Tensor(4.0, shape=(), dtype=float32)
3.0 4.0
node3: tf.Tensor(7.0, shape=(), dtype=float32)
node3.numpy(): 7.0


3.2 Placeholder

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np

# 2개의 값을 더하는 function을 정의합니다.
@tf.function
def add_two_values(x, y):
  return x + y

# 세션을 열고 그래프를 실행합니다.
# 출력값 :
# 7.5
# [ 3.  7.]
print(add_two_values(3, 4.5).numpy())
print(add_two_values(np.array([1, 3]), np.array([2, 4])).numpy())

# 노드를 추가해서 더 복잡한 그래프 형태를 만들어봅시다.
@tf.function
def add_two_values_and_multiply_three(x, y):
  return 3 * add_two_values(x, y)

# 출력값 : 22.5
print(add_two_values_and_multiply_three(3, 4.5).numpy())

7.5
[3 7]
22.5


3.3 Linear regression @tf2.0

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# 선형회귀 모델(Wx + b)을 위한 tf.Variable을 선언합니다.
W = tf.Variable(tf.random.normal(shape=[1]))
b = tf.Variable(tf.random.normal(shape=[1]))

@tf.function
def linear_model(x):
  return W*x + b

# 손실 함수를 정의합니다.
# MSE 손실함수 \mean{(y' - y)^2}
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.optimizers.SGD(0.01)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred = linear_model(x)
    loss = mse_loss(y_pred, y)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

# 트레이닝을 위한 입력값과 출력값을 준비합니다.
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]

# 경사하강법을 1000번 수행합니다.
for i in range(1000):
  train_step(x_train, y_train)

# 테스트를 위한 입력값을 준비합니다.
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형회귀 모델이 데이터의 경향성(y=2x)을 잘 학습했는지 측정합니다.
# 예상되는 참값 : [7, 10, 11, 12]
print(linear_model(x_test).numpy())

[ 7.0002875 10.001058  11.001314  12.001572 ]


3.4 Linear regression with tensorboard @tf2.0

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# 선형회귀 모델(Wx + b)을 위한 tf.Variable을 선언합니다.
W = tf.Variable(tf.random.normal(shape=[1]))
b = tf.Variable(tf.random.normal(shape=[1]))

@tf.function
def linear_model(x):
  return W*x + b

# 손실 함수를 정의합니다.
# MSE 손실함수 \mean{(y' - y)^2}
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.optimizers.SGD(0.01)

# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
summary_writer = tf.summary.create_file_writer('./tensorboard_log')

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred = linear_model(x)
    loss = mse_loss(y_pred, y)
    with summary_writer.as_default():
      tf.summary.scalar('loss', loss, step=optimizer.iterations)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

# 트레이닝을 위한 입력값과 출력값을 준비합니다.
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]

# 경사하강법을 1000번 수행합니다.
for i in range(1000):
  train_step(x_train, y_train)

# 테스트를 위한 입력값을 준비합니다.
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형회귀 모델이 데이터의 경향성(y=2x)을 잘 학습했는지 측정합니다.
# 예상되는 참값 : [7, 10, 11, 12]
print(linear_model(x_test).numpy())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[ 6.9964533  9.986951  10.983783  11.980616 ]


4.1 MNIST classification using softmax regression @tf2.0

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

# Softmax Regression 모델을 위한 tf.Variable들을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))

# Softmax Regression 모델을 정의합니다.
@tf.function
def softmax_regression(x):
  logits = tf.matmul(x, W) + b
  return tf.nn.softmax(logits)

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))
  #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logtis, labels=y)) # tf.nn.softmax_cross_entropy_with_logits API를 이용한 구현

# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.optimizers.SGD(0.5)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred = softmax_regression(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

# 1000번 반복을 수행하면서 최적화를 수행합니다.
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(batch_xs, batch_ys)

# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(softmax_regression(x_test), y_test)) # 정확도 : 약 91%

11493376/11490434 [==============================] - 0s 0us/step
정확도(Accuracy): 0.916400


4.2 tf_nn_sparse_softmax_cross_entropy_with_logits_example @tf2.0

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 레이블들을 int64 데이터 타입으로 변경합니다.
y_train, y_test = y_train.astype('int64'), y_test.astype('int64')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

# Softmax Regression 모델을 위한 tf.Variable들을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))

# Softmax Regression 모델을 정의합니다.
@tf.function
def softmax_regression(x):
  logits = tf.matmul(x, W) + b
  return tf.nn.softmax(logits), logits

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logtis, labels=y)) # tf.nn.softmax_cross_entropy_with_logits API를 이용한 구
  return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)) # tf.nn.sparse_softmax_cross_entropy_with_logits API를 이용한 구현현

# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), y)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.optimizers.SGD(0.5)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = softmax_regression(x)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

# 1000번 반복을 수행하면서 최적화를 수행합니다.
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(batch_xs, batch_ys)

# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(softmax_regression(x_test)[0], y_test)) # 정확도 : 약 91%

정확도(Accuracy): 0.916800


5.1 MNIST classification usiong ANN @tf2.0

In [0]:
# -*- coding: utf-8 -*-
# 텐서플로우를 이용한 ANN(Artificial Neural Networks) 구현

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

# 학습을 위한 설정값들을 정의합니다.
learning_rate = 0.001
num_epochs = 30     # 학습횟수
batch_size = 256    # 배치개수
display_step = 1    # 손실함수 출력 주기
input_size = 784    # 28 * 28
hidden1_size = 256
hidden2_size = 256
output_size = 10

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

# ANN 모델을 정의합니다.
class ANN(object):
  # ANN 모델을 위한 tf.Variable들을 정의합니다.
  def __init__(self):
    self.W1 = tf.Variable(tf.random.normal(shape=[input_size, hidden1_size]))
    self.b1 = tf.Variable(tf.random.normal(shape=[hidden1_size]))
    self.W2 = tf.Variable(tf.random.normal(shape=[hidden1_size, hidden2_size]))
    self.b2 = tf.Variable(tf.random.normal(shape=[hidden2_size]))
    self.W_output = tf.Variable(tf.random.normal(shape=[hidden2_size, output_size]))
    self.b_output = tf.Variable(tf.random.normal(shape=[output_size]))

  def __call__(self, x):
    H1_output = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
    H2_output = tf.nn.relu(tf.matmul(H1_output, self.W2) + self.b2)
    logits = tf.matmul(H2_output, self.W_output) + self.b_output

    return logits

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(learning_rate)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, vars(model).values())
  optimizer.apply_gradients(zip(gradients, vars(model).values()))

# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

# ANN 모델을 선언합니다.
ANN_model = ANN()

# 지정된 횟수만큼 최적화를 수행합니다.
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x, batch_y in train_data:
    # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    # 평균 손실을 측정합니다.
    average_loss += current_loss / total_batch
  # 지정된 epoch마다 학습결과를 출력합니다.
  if epoch % display_step == 0:
    print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

# 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test)) # 정확도: 약 94%

반복(Epoch): 1, 손실 함수(Loss): 312.994080
반복(Epoch): 2, 손실 함수(Loss): 65.030205
반복(Epoch): 3, 손실 함수(Loss): 41.822811
반복(Epoch): 4, 손실 함수(Loss): 30.622065
반복(Epoch): 5, 손실 함수(Loss): 23.496592
반복(Epoch): 6, 손실 함수(Loss): 18.703409
반복(Epoch): 7, 손실 함수(Loss): 15.184997
반복(Epoch): 8, 손실 함수(Loss): 12.452821
반복(Epoch): 9, 손실 함수(Loss): 10.281927
반복(Epoch): 10, 손실 함수(Loss): 8.480984
반복(Epoch): 11, 손실 함수(Loss): 6.901198
반복(Epoch): 12, 손실 함수(Loss): 5.684555
반복(Epoch): 13, 손실 함수(Loss): 4.647721
반복(Epoch): 14, 손실 함수(Loss): 3.825606
반복(Epoch): 15, 손실 함수(Loss): 3.095915
반복(Epoch): 16, 손실 함수(Loss): 2.493911
반복(Epoch): 17, 손실 함수(Loss): 2.023993
반복(Epoch): 18, 손실 함수(Loss): 1.567518
반복(Epoch): 19, 손실 함수(Loss): 1.269541
반복(Epoch): 20, 손실 함수(Loss): 0.997227
반복(Epoch): 21, 손실 함수(Loss): 0.768236
반복(Epoch): 22, 손실 함수(Loss): 0.556392
반복(Epoch): 23, 손실 함수(Loss): 0.447514
반복(Epoch): 24, 손실 함수(Loss): 0.322730
반복(Epoch): 25, 손실 함수(Loss): 0.244426
반복(Epoch): 26, 손실 함수(Loss): 0.179969
반복(Epoch): 27, 손실 함수(Loss): 0.131907
